One Year Historical Stock Quotes
===


In [1]:
import dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import pandas_datareader as data
from datetime import datetime as dt
import yfinance as yf
import numpy as np
import date_converter
global periods
periods = ''

In [2]:
print(f"pandas  version = {data.__version__}")

pandas  version = 0.7.0


In [3]:
# ดึงรายชื่อ ticker

def get_tickers():
    tick = pd.read_excel('data/world_market_symbols.xlsx',header=0)
    list1 = list(tick['shortname'])
    list2 = list(tick['symbol'])
    tickers = dict( zip( list1, list2))
    
    return tickers 

def tickers_map(value):
    tick = pd.read_excel('data/world_market_symbols.xlsx',header=0)
    list1 = list(tick['shortname'])
    list2 = list(tick['symbol'])
    tickers = dict( zip( list2, list1))    
    ticker = tickers[value]
    
    return ticker

def num_format(number,format_num=',.2f'):
    """
    Formatting helper - float (1 Item)
    """
    if np.isnan(number):
        return '-'
    return format(number, format_num)


In [4]:
PLOTLY_LOGO = "https://gitlab.com/yosarawut/financial-investment/raw/master/img/planet-earth.png?inline=false"

tickers = get_tickers()
dropdown = dbc.Row(
    [
        dbc.Col(dcc.Dropdown(
            id='my-dropdown',
            options=[{'label': s[0], 'value': str(s[1])}
                     for s in zip(tickers, tickers.values())],
            value='^GSPC',style={"margin-top": "10px", "margin-left": "10px", 
                                 "margin-right": "10px", "width": "300px"}))],


    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",align="center")
periods = '1y'
navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                    dbc.Col(dbc.NavbarBrand(
                        "Historical Stock Quotes", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://plot.ly",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(dropdown, id="navbar-collapse", navbar=True),
    ],
    color="primary",
    dark=True,
    className="mt-4",
)

In [5]:
button_group = dbc.ButtonGroup(
    [dbc.Button("1M", outline=True, size="sm", color="primary"), dbc.Button(
        "3M", outline=True,size="sm", color="primary"), dbc.Button("6M",outline=True,size="sm", color="primary"),
     dbc.Button("YTD",outline=True,size="md", color="primary"), dbc.Button("1Y",outline=True,size="md", color="primary"),
     dbc.Button("3Y",outline=True, color="primary"), dbc.Button("5Y",outline=True, color="primary"),  dbc.Button("10Y",outline=True, color="primary")],
style={"margin-top": "10px","margin-left": "10px","margin-right": "10px"})

graph =    dcc.Graph(id='my-graph', 
    style={"margin-left": "7%","margin-right": "auto","margin-top": "10px","width": "85%",'height':'350px'})

In [6]:
row = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(html.H2(id='ticker_name',className="display-5",style={"font-weight": "bold",'color':'#2A4B7C',
                                                                              "margin-left": "10px",}), md=6),
                dbc.Col(html.Div(""), md=2),
                dbc.Col(html.H2(id='regularMarketPrice',style={"text-align": "right",'color':'#2A4B7C',"font-weight": "bold"}), md=3),
                dbc.Col(html.Div(id='currency',style={"text-align": "left","margin-top": "18px",'color':'gray'}), md=1),
            ],
        style={"margin-top": "10px","margin-left": "10px","margin-right": "10px"}),
        dbc.Row(
            [
                dbc.Col(html.Div(id='lastdate',style={"text-align": "right"}),md=1),
                dbc.Col(html.Div(id="marketState",style={"text-align": "left","margin-left": "10px"}), md=2),
                dbc.Col(html.Div(""), md=5),
                dbc.Col(html.H4(id='pricechange',style={"text-align": "right","font-weight": "bold"}), md=2),
                dbc.Col(html.H4(id='percentchange',style={"text-align": "left","font-weight": "bold"}), md=2),
            ],
            style={"margin-top": "5px","margin-left": "10px","margin-right": "10px"}
        ),
    ]
)

In [7]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.layout = html.Div([navbar, row,graph])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    
    periods = '1y'    
    data = yf.Ticker(selected_dropdown_value)    
    df = data.history(period=periods)          
   
    return {
        'data': [{
            'x': df.index,
            'y': df.Close,            
        }],
        'layout': {'margin': {'l': 30, 'r': 0, 't': 20, 'b': 30}}       
    }

In [8]:
def get_info(tick):
    sym = yf.Ticker(tick)
    rep = sym.info
    return rep

In [9]:
@app.callback(Output('ticker_name', 'children'), [Input('my-dropdown', 'value')])
def update_ticker_children(selected_dropdown_value):      
    
    return tickers_map(selected_dropdown_value)


In [10]:
@app.callback(Output('regularMarketPrice', 'children'), [Input('my-dropdown', 'value')])
def update_regularMarketPrice_children(selected_dropdown_value):      
    a = get_info(selected_dropdown_value)
    price = num_format(a['regularMarketPrice'])
    
    return price

In [11]:
@app.callback(Output('currency', 'children'), [Input('my-dropdown', 'value')])
def update_currency_children(selected_dropdown_value):       
    a = get_info(selected_dropdown_value)    
    
    return a['currency']

In [12]:
@app.callback(Output('lastdate', 'children'), [Input('my-dropdown', 'value')])
def update_lastdate_children(selected_dropdown_value):      
    data = yf.Ticker(selected_dropdown_value)    
    df = data.history(period=periods) 
    last_date = max(df.index) 
    last_date = date_converter.date_to_string(last_date, '%d-%m-%Y')
    
    return last_date

In [13]:
@app.callback(Output('marketState', 'children'), [Input('my-dropdown', 'value')])
def update_marketState_children(selected_dropdown_value):     
    
    a = get_info(selected_dropdown_value)    
    
    return 'MARKET :  ' + a['marketState']

In [14]:
@app.callback(Output('pricechange', 'children'),[Input('my-dropdown', 'value')])
def update_pricechange_children(selected_dropdown_value):      
    data = yf.Ticker(selected_dropdown_value)    
    df = data.history(period=periods) 
    close = list(df.Close)
    x = np.array(close)
    start_price = x[0]
    current_price = x[-1]
    price_change = current_price - start_price
    price = num_format(price_change)
    if price_change < 0 :
        return html.Div(price, style={'color': 'red'})
    else:
        return html.Div('+ '+price, style={'color': '#2A4B7C'})   
   

In [15]:
@app.callback(Output('percentchange', 'children'),[Input('my-dropdown', 'value')])
def update_percentchange_children(selected_dropdown_value):   
    
    data = yf.Ticker(selected_dropdown_value)    
    df = data.history(period=periods) 
    close = list(df.Close)
    x = np.array(close)
    start_price = x[0]
    current_price = x[-1]
    price_change = current_price - start_price
    percent_change = (price_change / start_price)
    percent = num_format(percent_change*100)
    percent = (percent  + ' %)'   )   
    if percent_change < 0 :
        return html.Div('( '+percent, style={'color': 'red'})
    else:
        return html.Div('(+ '+percent, style={'color': '#2A4B7C'})

In [16]:
app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
D:\Anaconda\lib\site-packages\dash\resources.py:44: UserWarning:

A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available

127.0.0.1 - - [11/Oct/2019 17:21:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:21:19] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:21:19] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:21:19] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:21:19] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1561063963 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:21:19] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563307603 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oc

127.0.0.1 - - [11/Oct/2019 17:24:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2019 17:24:50] "POST /_dash-update-component HTTP/1.1" 200 -
